In [168]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import max_error
from sklearn.datasets import make_regression
import numpy as np


In [108]:
df = pd.read_csv('flight.csv')
test = pd.read_csv('test.csv')

In [109]:
avg_delay_origin = df[['origin','arr_delay']].groupby(['origin'])['arr_delay'].mean().sort_values()
test['origin_delays']=test['origin']
test['origin_delays'] = test['origin_delays'].replace(avg_delay_origin)

In [110]:
avg_delay_dest = df[['dest','arr_delay']].groupby(['dest'])['arr_delay'].mean().sort_values()
test['dest_delays']=test['dest']
test['dest_delays'] = test['dest_delays'].replace(avg_delay_dest)

In [111]:
# delay_by_month = df[['Month','arr_delay']].groupby(['Month'])['arr_delay'].mean().sort_values()
# df['delay_by_month']= df['Month']
# df['delay_by_month'] = df['delay_by_month'].replace(delay_by_month)
# df['delay_by_month']
#Realized my month feature is pretty useless since we are only predicting a week

In [112]:
# delay_by_day = df[['Day','arr_delay']].groupby(['Day'])['arr_delay'].mean().sort_values()
# df['delay_by_day']= df['Day']
# df['delay_by_day'] = df['delay_by_day'].replace(delay_by_day)
# df['delay_by_day']
#my day feature is pretty useless since we are only predicting 7 days

In [154]:
def split_data(test, drop_list):
    test = test.drop(drop_list,axis=1)
    print(test.columns)
    #test train split time
    from sklearn.model_selection import train_test_split
    y = test['airline_delay'].values #target
    X = test.drop(['fl_date', 'mkt_unique_carrier', 'mkt_carrier_fl_num', 'origin', 'dest'],axis=1).values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=100)

    print("train-set size: ", len(y_train),
      "\ntest-set size: ", len(y_test))
    return X_train, X_test, y_train, y_test

In [114]:
def get_predictions(clf, X_train, y_train, X_test):
    # create classifier
    clf = clf
    # fit it to training data
    clf.fit(X_train,y_train)
    # predict using test data
    y_pred = clf.predict(X_test)
    return y_pred

In [115]:
#Function to print the regressor's scores
def print_scores(y_test,y_pred):
    print('Mean Absolute Error: ', mean_absolute_error(y_test, y_pred))
    print('R2: ', r2_score(y_test, y_pred))
    print('Max Error: ', max_error(y_test, y_pred))

In [116]:
# test.drop(test[(test['dest']=='SMX')].index, inplace=True)
# test.drop(test[(test['origin']=='SMX')].index, inplace=True)
# test.drop(test[(test['dest']=='OWB')].index, inplace=True)
# test.drop(test[(test['origin']=='OWB')].index, inplace=True)
# test.drop(test[(test['dest']=='BFM')].index, inplace=True)
# test.drop(test[(test['origin']=='BFM')].index, inplace=True)
# test.drop(test[(test['dest']=='OGS')].index, inplace=True)
# test.drop(test[(test['origin']=='OGS')].index, inplace=True)

In [117]:
avg_delay_airline = df[['mkt_unique_carrier','arr_delay']].groupby(['mkt_unique_carrier'])['arr_delay'].mean().sort_values()
test['airline_delay']=test['mkt_unique_carrier']
test['airline_delay'] = test['airline_delay'].replace(avg_delay_airline)

In [118]:
data1 = ['origin_delays','dest_delays']


In [155]:
X_train, X_test, y_train, y_test = split_data(test, data1)
y_pred = get_predictions(RandomForestRegressor(), X_train, y_train, X_test)
print_scores(y_test,y_pred)

Index(['fl_date', 'mkt_unique_carrier', 'mkt_carrier_fl_num', 'origin', 'dest',
       'distance', 'airline_delay', 'predict_delay'],
      dtype='object')
train-set size:  120498 
test-set size:  30125
Mean Absolute Error:  1.3578669217152677e-13
R2:  1.0
Max Error:  2.2026824808563106e-13


In [158]:
reg = RandomForestRegressor()
reg.fit(X_train, y_train)
# y_pred.fit(X_train, y_train)

RandomForestRegressor()

In [170]:
y_pred = reg.predict(X_test)
y_predd = reg.predict(X_train)
y_pred = np.concatenate((y_pred,y_predd),axis=0)

In [174]:
test['predict_delay'] = y_pred

In [176]:
test.drop(columns=['origin_delays','dest_delays','airline_delay'])

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,distance,predict_delay
0,2020-01-01,WN,5888,ONT,SFO,363,3.205575
1,2020-01-01,WN,6276,ONT,SFO,363,2.273931
2,2020-01-01,WN,4598,ONT,SJC,333,8.736684
3,2020-01-01,WN,4761,ONT,SJC,333,6.170646
4,2020-01-01,WN,5162,ONT,SJC,333,3.205575
...,...,...,...,...,...,...,...
150618,2020-01-07,DL,4813,DTW,JFK,509,2.273931
150619,2020-01-07,DL,4814,GSP,LGA,610,0.345636
150620,2020-01-07,DL,4815,ATL,XNA,589,3.205575
150621,2020-01-07,DL,4815,XNA,ATL,589,8.736684


In [177]:
test.to_csv('final_result.csv')